## 1. Download CityGML files form OGD Vienna

https://www.wien.gv.at/ma41datenviewer/public/start.aspx

Select **Generalisiertes Dachmodell (LOD2.1)** and download areas of interest to get CityGML files.




## 2. Process CityGML-files

Edit GML files with GMLProcesser class:
- gml files from OGD Vienna are missing address information
- calculate centroid of building footprint
- convert centroid coordinates to WRG84 (OSM) to get correct address information
- get address information from OSM
- extend building elements in GML with xAL address elements
- change building name to address for better identification

In [1]:
import os
import sys
from pathlib import Path
sys.path.append(str(Path().absolute().parent))

from dhc_sim.processing.citygml import GMLProcessor
from dhc_sim.utils import find_files_by_extension

current_dir = os.getcwd()

path = os.path.join(current_dir, 'data', 'citygml_files', 'raw_ogd_vienna')
files = find_files_by_extension(path, 'gml')

output_folder = os.path.join(current_dir, 'data', 'citygml_files', 'preprocessed_citygml')

for file in files:
    file_name = file['file_name']
    input_path = file['path']
    output_path = os.path.join(output_folder, file_name)
    
    print(f"Initializing GML Address Processor for {file_name}")
    processor = GMLProcessor(input_path)
    
    # Process all buildings
    successful_buildings, failed_buildings = processor.process_all_buildings()
    
    # Save the modified file
    processor.save(output_path)
    
    # Print processing statistics
    print("\nProcessing completed!")
    print(f"Successfully processed: {successful_buildings}")
    print(f"Failed to process: {failed_buildings}")
    print(f"Output file saved as: {output_path}")


    

Initializing GML Address Processor for 100087.gml
Found 132 buildings
Creating GeoDataFrame and calculating building footprints...
Created GeoDataFrame with 132 rows
Processing building 1/132
Processing building 2/132
Processing building 3/132
Processing building 4/132
Processing building 5/132
Processing building 6/132
Processing building 7/132
Processing building 8/132
Processing building 9/132
Processing building 10/132
Processing building 11/132
Processing building 12/132
Processing building 13/132
Processing building 14/132
Processing building 15/132
Processing building 16/132
Processing building 17/132
Processing building 18/132
Processing building 19/132
Processing building 20/132
Processing building 21/132
Processing building 22/132
Processing building 23/132
Processing building 24/132
Processing building 25/132
Processing building 26/132
Processing building 27/132
Processing building 28/132
Processing building 29/132
Processing building 30/132
Processing building 31/132
Proces

## 3. Import GMLs in 3DCityDB
- open 3D City Database Importer/Exporter
- connect to DB
- select preprocessed GMLs an import in DB

**Importer Console output:**
```
[13:10:41 INFO] Initializing database import...
[13:10:42 INFO] Spatial indexes are enabled.
[13:10:42 INFO] Normal indexes are enabled.
[13:10:42 INFO] Creating list of files to be imported...
[13:10:43 INFO] List of import files successfully created.
[13:10:43 INFO] 9 file(s) will be imported.
[13:11:52 INFO] Imported city objects:
[13:11:52 INFO] bldg:Building: 1507
[13:11:52 INFO] bldg:BuildingPart: 3931
[13:11:52 INFO] bldg:GroundSurface: 4463
[13:11:52 INFO] bldg:RoofSurface: 16735
[13:11:52 INFO] bldg:WallSurface: 55325
[13:11:52 INFO] Processed geometry objects: 392833
[13:11:52 INFO] Total import time: 01 m, 11 s.
[13:11:52 INFO] Database import successfully finished.
```

## 4. Connect QGIS to 3DCityDB

### Establish DB-Connection
- Install "3DCityDB Tool for QGIS" plugin
- Open 3DCityDB admin panel
- Install server-side QGIS Package to the database
- Add Database user to QGIS Package user group
- Create DB schema for user-related QGIS data


<img src="./pics/01_04_01.png" width="400">

### Load 3DCityDB Layer

- select db connection
- connect to db
- create layers for schema 'citydb' and refresh layers (only needs to be done at first import)
- select boundary box to import only area of interest



## 5. Edit citygml features of buildings in QGIS

**Calculate storeys above ground**
- citygml from OGD Vienna doesn't countain information about storey heights and storey numbers
- calculate parameter "storeys_above_ground" with QGIS field calculater
    - formular:
    
        ```python
            if(floor(Height / 3.6) < 1, 1, floor(Height / 3.6))
        ```
    - floor function = round down
    - 3.6 = average height of floor in m
- calculate parameter "storey_height_above_ground"
    - formular:
    
        ```python
            round("measured_height" / "storeys_above_ground", 2)
        ```
- set parameter "storey_height_ag_unit" to m
- set parameter "storeys_below_ground" to 0
- set parameter "storey_height_below_ground" to 0
- set parameter "storey_height_bg_unit" to m

**Set Year of construction and usage**
- set year of construction for all buildings to 1908 (only for demonstration)
- set Class = "1000" --> Habitat
- set Function = "1000" --> Residential Building
- set Usage = "1000" --> Residential


## 6. Export area of interest as citygml file

**Add generict attribute in 3DCityDB**
- add generic attribute "teaser_export" to flag buildings for teaser export
- for every cityobject set attr "teaser_export" to false

    ```sql
    INSERT INTO cityobject_genericattrib (attrname, datatype, strval, cityobject_id)
    SELECT 'teaser_export', 1, 'false', id
    FROM cityobject;
    ```
**Flag buildings for export in QGIS**
- select buildings and edit generic building attribute "teaser_export"
- set attribute to "true"
- **Problem:** no bulk edit of generic attributes in QGIS possible, attribute has to be set one by one
- **Solution:**
    - either: set description to "teaser_export=true"
    - or: set class to "1000" -> set building class only on buildings you want to export
    - use one of these as flags

**Export buildings with 3DCityDB Importer/Exporter**

1. Export via flag in building description:
    ```sql
    SELECT id
    FROM cityobject
    WHERE description="teaser_export=true"
    ```
    **Problem:** using this query in the 3DCityDB Importer/Exporter only exports building but not buildingparts (subelements of building).
    trying it in the pgAdmin it outputs the correct cityobject ids for both building and buildingparts.
    

2. Export via flag in building class:
    ```sql
    SELECT building_root_id
    FROM building
    WHERE class='1000'
    ```
    working fine but not a clean solution